## This is following the tutorial from
https://mpb.readthedocs.io/en/latest/Python_Tutorial/

In [1]:
import math
import meep as mp
from meep import mpb


In [2]:
boundary_k_points = [mp.Vector3(),               # Gamma
              mp.Vector3(y=0.5),          # M
              mp.Vector3(-1 / 3, 1 / 3),  # K
              mp.Vector3()]               # Gamma

k_points = mp.interpolate(4, boundary_k_points)
num_bands = 8
geometry = [mp.Cylinder(0.2, material=mp.Medium(epsilon=12))]
geometry_lattice = mp.Lattice(size=mp.Vector3(1, 1),
                                 basis1=mp.Vector3(math.sqrt(3) / 2, 0.5),
                                 basis2=mp.Vector3(math.sqrt(3) / 2, -0.5))
resolution = 32
ms = mpb.ModeSolver(num_bands=num_bands,
                    k_points=k_points,
                    geometry=geometry,
                    geometry_lattice=geometry_lattice,
                    resolution=resolution)

ms.run_tm()

Initializing eigensolver data
Computing 8 bands with 1e-07 tolerance
Working in 2 dimensions.
Grid size is 32 x 32 x 1.
Solving for 8 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (0.866025, 0.5, 0)
     (0.866025, -0.5, 0)
     (0, 0, 1)
Cell volume = 0.866025
Reciprocal lattice vectors (/ 2 pi):
     (0.57735, 1, -0)
     (0.57735, -1, 0)
     (-0, 0, 1)
Geometric objects:
     cylinder, center = (0,0,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
Geometric object tree has depth 1 and 1 object nodes (vs. 1 actual objects)
Initializing epsilon function...
Allocating fields...
Solving for band polarization: tm.
Initializing fields to random numbers...
16 k-points
  Vector3<0.0, 0.0, 0.0>
  Vector3<0.0, 0.1, 0.0>
  Vector3<0.0, 0.2, 0.0>
  Vector3<0.0, 0.30000000000000004, 0.0>
  Vector3<0.0, 0.4, 0.0>
  Vector3<0.0, 0.5, 0.0>
  Vector3<-0.06666666666666667, 0.4666666666666667, 0.0>
  Vector3<-0.13333333333333333, 0.43333333333333335, 0.0>
  Vect

In [3]:
def first_tm_gap(r):
    ms.geometry = [mp.Cylinder(r, material=mp.Medium(epsilon=12))]
    ms.run_tm()
    
    return -1 * ms.retrieve_gap(1) # return the gap from TM band 1 to TM band 2


In [4]:
ms.num_bands = 2
ms.mesh_size = 7


In [5]:
from scipy.optimize import minimize_scalar

result = minimize_scalar(first_tm_gap, method='bounded', bounds=[0.1, 0.5], options={'xatol': 0.1})
print("radius at maximum: {}".format(result.x))
print("gap size at maximum: {}".format(result.fun * -1))

Initializing eigensolver data
Computing 2 bands with 1e-07 tolerance
Working in 2 dimensions.
Grid size is 32 x 32 x 1.
Solving for 2 bands at a time.
Creating Maxwell data...
Mesh size is 7.
Lattice vectors:
     (0.866025, 0.5, 0)
     (0.866025, -0.5, 0)
     (0, 0, 1)
Cell volume = 0.866025
Reciprocal lattice vectors (/ 2 pi):
     (0.57735, 1, -0)
     (0.57735, -1, 0)
     (-0, 0, 1)
Geometric objects:
     cylinder, center = (0,0,0)
          radius 0.252786, height 1e+20, axis (0, 0, 1)
Geometric object tree has depth 1 and 1 object nodes (vs. 1 actual objects)
Initializing epsilon function...
Allocating fields...
Solving for band polarization: tm.
Initializing fields to random numbers...
16 k-points
  Vector3<0.0, 0.0, 0.0>
  Vector3<0.0, 0.1, 0.0>
  Vector3<0.0, 0.2, 0.0>
  Vector3<0.0, 0.30000000000000004, 0.0>
  Vector3<0.0, 0.4, 0.0>
  Vector3<0.0, 0.5, 0.0>
  Vector3<-0.06666666666666667, 0.4666666666666667, 0.0>
  Vector3<-0.13333333333333333, 0.43333333333333335, 0.0>
 

In [6]:
ms.mesh_size = 3

As another example, let's construct a structure with a complete 2d gap (i.e., in both TE and TM polarizations), in a somewhat unusual way: using a dielectric structure. An anisotropic dielectric presents a different dielectric constant depending upon the direction of the electric field, and can be used in this case to make the TE and TM polarizations "see" different structures.

We already know that the triangular lattice of rods has a gap for TM light, but not for TE light. The dual structure, a triangular lattice of holes, has a gap for TE light but not for TM light at least for the small radii we will consider. Using an anisotropic dielectric, we can make both of these structures simultaneously, with each polarization seeing the structure that gives it a gap.



In [7]:
ms.geometry = [mp.Cylinder(radius=0.3, material=mp.Medium(epsilon_diag=mp.Vector3(1, 1, 12)))]
ms.default_material = mp.Medium(epsilon_diag=mp.Vector3(12, 12, 1))
ms.num_bands = 8
ms.run()  # just use run, instead of run_te or run_tm, to find the complete gap

Initializing eigensolver data
Computing 8 bands with 1e-07 tolerance
Working in 2 dimensions.
Grid size is 32 x 32 x 1.
Solving for 8 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (0.866025, 0.5, 0)
     (0.866025, -0.5, 0)
     (0, 0, 1)
Cell volume = 0.866025
Reciprocal lattice vectors (/ 2 pi):
     (0.57735, 1, -0)
     (0.57735, -1, 0)
     (-0, 0, 1)
Geometric objects:
     cylinder, center = (0,0,0)
          radius 0.3, height 1e+20, axis (0, 0, 1)
Geometric object tree has depth 1 and 1 object nodes (vs. 1 actual objects)
Initializing epsilon function...
Allocating fields...
Solving for band polarization: .
Initializing fields to random numbers...
16 k-points
  Vector3<0.0, 0.0, 0.0>
  Vector3<0.0, 0.1, 0.0>
  Vector3<0.0, 0.2, 0.0>
  Vector3<0.0, 0.30000000000000004, 0.0>
  Vector3<0.0, 0.4, 0.0>
  Vector3<0.0, 0.5, 0.0>
  Vector3<-0.06666666666666667, 0.4666666666666667, 0.0>
  Vector3<-0.13333333333333333, 0.43333333333333335, 0.0>
  Vector

In [9]:
ms.geometry_lattice = mp.Lattice(size=mp.Vector3(5, 5))
ms.geometry = [mp.Cylinder(0.2, material=mp.Medium(epsilon=12))] # specify the initial geometry
ms.geometry = mp.geometric_objects_lattice_duplicates(ms.geometry_lattice, ms.geometry) #duplicate the geometry over lattice points

ms.geometry.append(mp.Cylinder(0.2, material=mp.air))
ms.k_points = [mp.Vector3(0.5, 0.5)]
ms.num_bands = 50


In [10]:
ms.run_tm()

Initializing eigensolver data
Computing 50 bands with 1e-07 tolerance
Working in 2 dimensions.
Grid size is 160 x 160 x 1.
Solving for 10 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (5, 0, 0)
     (0, 5, 0)
     (0, 0, 1)
Cell volume = 25
Reciprocal lattice vectors (/ 2 pi):
     (0.2, -0, 0)
     (-0, 0.2, -0)
     (0, -0, 1)
Geometric objects:
     cylinder, center = (2,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (1,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (0,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-1,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-2,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (2,1,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (1,1,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (0,1,0)
 

In [11]:
mpb.output_efield_z(ms,25)

Outputting fields to e.k01.b25.z.tm.h5...


In [13]:
ms.get_dfield(25)  # compute the D field for band 25
ms.compute_field_energy()  # compute the energy density from D
c = mp.Cylinder(1.0, material=mp.air)
print("energy in cylinder: {}".format(ms.compute_energy_in_objects([c])))

D-energy-components:, 1, 25, 0, 0, 1
energy in cylinder: 0.6264485251071318


In [14]:
ms.num_bands = 1  # only need to compute a single band, now!
ms.target_freq = (0.2812 + 0.4174) / 2
ms.tolerance = 1e-8

In [15]:
ms.run_tm()

Initializing eigensolver data
Computing 1 bands with 1e-08 tolerance
Target frequency is 0.3493
Working in 2 dimensions.
Grid size is 160 x 160 x 1.
Solving for 1 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (5, 0, 0)
     (0, 5, 0)
     (0, 0, 1)
Cell volume = 25
Reciprocal lattice vectors (/ 2 pi):
     (0.2, -0, 0)
     (-0, 0.2, -0)
     (0, -0, 1)
Geometric objects:
     cylinder, center = (2,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (1,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (0,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-1,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-2,2,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (2,1,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cylinder, center = (1,1,0)
          radius 0.2, height 1e+20, axis (0, 0, 1)
     cyl